In [1]:
import re
import json
import numpy as np 
import pandas as pd
from loguru import logger
from modules import _get_driver_webscraping, _get_headers_webscraping, _number_of_pages, _get_html_with_many_cards, _read_results_parquet, _apply_functions_in_parallel_to_webscraping, _read_json_file, _extract_data_in_html_for_page_with_many_cards, _get_html_especifc_for_wich_property, _transform_data_to_dataframe

In [2]:
base_url = 'https://www.zapimoveis.com.br'
transaction = "aluguel"
type = "imoveis"
local = "rj+rio-de-janeiro"

In [3]:
# number_of_pages = _number_of_pages(base_url, transaction, type, local)
# print(number_of_pages)

number_of_pages = {'Parametros': {'Transacao': 'aluguel', 'Tipo': 'imoveis', 'Local': 'rj+rio-de-janeiro'}, 'Requisicao': {'Status': True}, 'Imoveis': 26981, 'Paginas': 269}
number_of_pages = json.dumps(number_of_pages, indent=4)
print(number_of_pages)


{
    "Parametros": {
        "Transacao": "aluguel",
        "Tipo": "imoveis",
        "Local": "rj+rio-de-janeiro"
    },
    "Requisicao": {
        "Status": true
    },
    "Imoveis": 26981,
    "Paginas": 269
}


In [1]:
# list_use_in_function = list(range(1, 269))
# function_to_apply = _get_html_with_many_cards
# name_file = 'urls_imoveis_aluguel'
# workers = 3
# args = [base_url, transaction, type, local]

# info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)
# info_pages, erro_pages

In [3]:
lista_informacoes_imoveis, erros_imoveis = _read_results_parquet(key_name="url", name_file="urls_imoveis_aluguel")
logger.success(f"Informações dos Imóveis Processadas: {len(lista_informacoes_imoveis)}")

2024-11-13 07:39:48.401 | SUCCESS  | __main__:<module>:2 - Informações dos Imóveis Processadas: 268


In [4]:
dataframe_urls = _extract_data_in_html_for_page_with_many_cards(lista_informacoes_imoveis)
dataframe_urls.head()
dataframe_urls = dataframe_urls.dropna(subset=['url_imovel'])
print(dataframe_urls.shape)
dataframe_urls.head()

(10064, 9)


,pagina,anuncio_pagina,id_imovel,url_imovel,destaque,bairro,endereco,descricao,data_completa
0,0,0,2741566013,https://www.zapimoveis.com.br/imovel/aluguel-c...,Super Destaque,"Laranjeiras, Rio de Janeiro",Rua Alice,"Aliança Viva aluga: Casa na Rua Alice, em Lara...",2024-11-13
1,0,1,2729441818,https://www.zapimoveis.com.br/imovel/aluguel-l...,Super Destaque,"Barra da Tijuca, Rio de Janeiro",Avenida das Américas,Ótima oportunidade para quem busca um espaço c...,2024-11-13
2,0,2,2750169530,https://www.zapimoveis.com.br/imovel/aluguel-a...,Super Destaque,"Lagoa, Rio de Janeiro",Avenida Epitácio Pessoa,"(Código: 36354, Avenida Epitácio Pessoa) - Loc...",2024-11-13
3,0,3,2755807416,https://www.zapimoveis.com.br/imovel/aluguel-c...,Destaque,"Copacabana, Rio de Janeiro",Rua Francisco Sá,Monte seu consultório/sala de pilates AQUI!\n\...,2024-11-13
5,0,5,2750819430,https://www.zapimoveis.com.br/imovel/venda-and...,NaN,"Centro, Rio de Janeiro",Avenida Marechal Floriano,"Centro, RJ. Conjunto comercial medindo aproxim...",2024-11-13


In [5]:
urls_de_imoveis = dataframe_urls['url_imovel'].to_list()
logger.success(f"URLs Processadas: {len(urls_de_imoveis)}")

2024-11-13 07:41:05.681 | SUCCESS  | __main__:<module>:2 - URLs Processadas: 10064


In [9]:
#list_use_in_function = urls_de_imoveis
#function_to_apply = _get_html_especifc_for_wich_property
#name_file = 'caracteristicas_imoveis_aluguel'
#workers = 3
#args = []

#info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)

In [2]:
lista_informacoes_imoveis_aluguel, erros_imoveis_aluguel = _read_results_parquet(key_name="url", name_file="data/caracteristicas_imoveis_aluguel")
logger.success(f"Informações dos Imóveis Processadas: {len(lista_informacoes_imoveis_aluguel)}")

2024-11-23 16:12:59.497 | SUCCESS  | __main__:<module>:2 - Informações dos Imóveis Processadas: 8322


In [3]:
dados_imoveis = _transform_data_to_dataframe(lista_informacoes_imoveis_aluguel, aluguel=True)

print(dados_imoveis.shape)
dados_imoveis.head()

(8322, 19)


,url,endereco,preco,area,andar,quartos,suites,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,NaN,3.0,1.0,2,1982.0,470.0,False,False,False,False,True,False,False,residencial,apartamento
1,https://www.zapimoveis.com.br/imovel/aluguel-c...,"Rua Alice - Laranjeiras, Rio de Janeiro - RJ",20000.0,300.0,NaN,8.0,2.0,4,NaN,6558.0,False,False,False,False,False,False,False,residencial,casa
2,https://www.zapimoveis.com.br/imovel/aluguel-l...,"Avenida das Américas, 5001 - Barra da Tijuca, ...",6000.0,70.0,1.0,NaN,NaN,2,2940.0,20933.0,False,False,False,True,False,False,False,comercial,loja
3,https://www.zapimoveis.com.br/imovel/aluguel-c...,"Rua Francisco Sá, 23 - Copacabana, Rio de Jane...",1350.0,35.0,9.0,NaN,NaN,1,1080.0,258.0,False,False,False,False,False,False,False,comercial,conjunto
4,https://www.zapimoveis.com.br/imovel/venda-and...,"Avenida Marechal Floriano, 19 - Centro, Rio de...",8308000.0,1923.0,NaN,NaN,NaN,0,54700.0,10450.0,False,False,False,False,False,False,False,comercial,laje


In [5]:
dados_imoveis.to_parquet("dataframe_imoveis_aluguel_tratados.parquet", index=False)